In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, roc_curve, precision_score, recall_score, f1_score, accuracy_score
from os import cpu_count
from math import floor
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
# from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB
# import shap
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
# shap.initjs()

In [ ]:
data=pd.read_csv("../dataset/NF_TON_IoT_V2/NF-ToN-IoT-v2.csv")

In [ ]:
data.head()

,L4_SRC_PORT,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label,Attack
0,-2017,80,6,7.0,140,1,0,0,2,2,...,512,0,0,0,0,0,0,0.0,1,DoS
1,-4762,80,6,7.0,280,2,0,0,2,2,...,512,0,0,0,0,0,0,0.0,1,DoS
2,32534,80,6,7.0,280,2,0,0,2,2,...,512,0,0,0,0,0,0,0.0,1,DoS
3,-13423,80,17,188.0,56,2,0,0,0,0,...,0,0,0,0,0,0,0,0.0,1,DDoS
4,-28756,80,17,188.0,56,2,0,0,0,0,...,0,0,0,0,0,0,0,0.0,1,DDoS


In [ ]:
data.dtypes

L4_SRC_PORT                      int16
L4_DST_PORT                      int16
PROTOCOL                          int8
L7_PROTO                       float32
IN_BYTES                         int32
IN_PKTS                          int32
OUT_BYTES                        int32
OUT_PKTS                         int32
TCP_FLAGS                        int16
CLIENT_TCP_FLAGS                 int16
SERVER_TCP_FLAGS                 int16
FLOW_DURATION_MILLISECONDS       int32
DURATION_IN                      int16
DURATION_OUT                     int16
MIN_TTL                          int16
MAX_TTL                          int16
LONGEST_FLOW_PKT                 int32
SHORTEST_FLOW_PKT                int16
MIN_IP_PKT_LEN                   int16
MAX_IP_PKT_LEN                   int32
SRC_TO_DST_SECOND_BYTES        float32
DST_TO_SRC_SECOND_BYTES        float32
RETRANSMITTED_IN_BYTES           int32
RETRANSMITTED_IN_PKTS            int16
RETRANSMITTED_OUT_BYTES          int16
RETRANSMITTED_OUT_PKTS   

In [ ]:
data.Label.value_counts()

1    30290649
0      129437
Name: Label, dtype: int64

In [ ]:
data.Attack.value_counts()

DDoS              14280259
DoS               13645057
Reconnaissance     2363017
Benign              129437
Theft                 2316
Name: Attack, dtype: int64

In [ ]:
data=data.drop(columns=['L4_SRC_PORT', 'L4_DST_PORT']) #dropping metadata

In [ ]:
training_set = data.sample(frac=0.05, replace=False,random_state=42)
# 1%train, 99% test
testing_set = data.drop(index=training_set.index)

In [ ]:
training_set.Attack.value_counts()

DDoS              713308
DoS               682874
Reconnaissance    118292
Benign              6425
Theft                105
Name: Attack, dtype: int64

In [ ]:
attacks=training_set.Attack.unique()
attacks=['Benign','Reconnaissance', 'DDoS', 'DoS', 'Theft']

In [ ]:
corr = training_set.corr()
corr_features={corr.columns[i] : corr.columns[(corr>0.9).iloc[i]].values.tolist() for i in range(0,corr.shape[0])}
corr_list=[]
for key,value in corr_features.items():
#     check if we already have this set
    have_set=False
    for set_s in corr_list:
        if key in set_s:
#             we have found a set
            have_set=True
            break
    if have_set==False and len(value)>1:
        corr_list.append(value)

In [ ]:
corr_list

[['PROTOCOL', 'L7_PROTO'],
 ['IN_BYTES',
  'IN_PKTS',
  'NUM_PKTS_512_TO_1024_BYTES',
  'NUM_PKTS_1024_TO_1514_BYTES'],
 ['OUT_BYTES', 'OUT_PKTS', 'NUM_PKTS_1024_TO_1514_BYTES'],
 ['TCP_FLAGS', 'SERVER_TCP_FLAGS', 'MIN_IP_PKT_LEN'],
 ['MIN_TTL', 'MAX_TTL'],
 ['LONGEST_FLOW_PKT', 'MAX_IP_PKT_LEN'],
 ['RETRANSMITTED_IN_BYTES', 'RETRANSMITTED_IN_PKTS'],
 ['RETRANSMITTED_OUT_BYTES', 'RETRANSMITTED_OUT_PKTS'],
 ['ICMP_TYPE', 'ICMP_IPV4_TYPE']]

In [ ]:
#correction because NUM_PKTS_1024_TO_1514_BYTES appears twice
corr_list[2]=corr_list[2][:-1]
corr_list

[['PROTOCOL', 'L7_PROTO'],
 ['IN_BYTES',
  'IN_PKTS',
  'NUM_PKTS_512_TO_1024_BYTES',
  'NUM_PKTS_1024_TO_1514_BYTES'],
 ['OUT_BYTES', 'OUT_PKTS'],
 ['TCP_FLAGS', 'SERVER_TCP_FLAGS', 'MIN_IP_PKT_LEN'],
 ['MIN_TTL', 'MAX_TTL'],
 ['LONGEST_FLOW_PKT', 'MAX_IP_PKT_LEN'],
 ['RETRANSMITTED_IN_BYTES', 'RETRANSMITTED_IN_PKTS'],
 ['RETRANSMITTED_OUT_BYTES', 'RETRANSMITTED_OUT_PKTS'],
 ['ICMP_TYPE', 'ICMP_IPV4_TYPE']]

In [ ]:
pip install torch-geometric

In [ ]:
pip install torch torchvision torchaudio

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.loader import DataLoader
import networkx as nx
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)


AttributeError: partially initialized module 'torch_geometric' has no attribute 'typing' (most likely due to a circular import)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.loader import DataLoader
import networkx as nx
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# 1. Data Loading and Preprocessing
def load_and_preprocess_data(file_path):
    # Load data
    data = pd.read_parquet(file_path)
    
    # Handle negative port values
    data['L4_SRC_PORT'] = data['L4_SRC_PORT'].abs()
    data['L4_DST_PORT'] = data['L4_DST_PORT'].abs()
    
    # Select numerical features
    numerical_features = [
        'L4_SRC_PORT', 'L4_DST_PORT', 'IN_BYTES', 'IN_PKTS', 
        'OUT_BYTES', 'OUT_PKTS', 'TCP_WIN_MAX_IN', 'TCP_WIN_MAX_OUT'
    ]
    
    # Handle missing values
    data[numerical_features] = data[numerical_features].fillna(0)
    
    # Encode categorical variables
    le_attack = LabelEncoder()
    data['Attack_encoded'] = le_attack.fit_transform(data['Attack'])
    
    # Standardize numerical features
    scaler = StandardScaler()
    data[numerical_features] = scaler.fit_transform(data[numerical_features])
    
    return data, numerical_features, le_attack

# 2. Exploratory Data Analysis
def perform_eda(data, numerical_features):
    print("\nEDA Summary:")
    print("\nDataset Shape:", data.shape)
    print("\nMissing Values:\n", data.isnull().sum().sum())
    print("\nLabel Distribution:\n", data['Label'].value_counts())
    print("\nAttack Type Distribution:\n", data['Attack'].value_counts())
    
    # Correlation heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(data[numerical_features].corr(), annot=True, cmap='coolwarm')
    plt.title('Feature Correlation Heatmap')
    plt.savefig('correlation_heatmap.png')
    plt.close()
    
    # Attack type distribution plot
    plt.figure(figsize=(10, 6))
    sns.countplot(x='Attack', data=data)
    plt.xticks(rotation=45)
    plt.title('Attack Type Distribution')
    plt.savefig('attack_distribution.png')
    plt.close()

# 3. Graph Construction
def create_graph_data(data, numerical_features, target_col='Label'):
    # Create edge index based on similar source/destination ports
    edge_index = []
    for i in range(len(data)):
        for j in range(i+1, len(data)):
            if (data.iloc[i]['L4_SRC_PORT'] == data.iloc[j]['L4_SRC_PORT'] or 
                data.iloc[i]['L4_DST_PORT'] == data.iloc[j]['L4_DST_PORT']):
                edge_index.append([i, j])
                edge_index.append([j, i])
    
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    
    # Node features
    x = torch.tensor(data[numerical_features].values, dtype=torch.float)
    
    # Labels
    y = torch.tensor(data[target_col].values, dtype=torch.long)
    
    # Create PyG data object
    graph_data = Data(x=x, edge_index=edge_index, y=y)
    
    # Create train/test mask
    train_mask, test_mask = train_test_split(
        range(len(data)), test_size=0.2, random_state=42, stratify=data[target_col]
    )
    graph_data.train_mask = torch.tensor(train_mask, dtype=torch.long)
    graph_data.test_mask = torch.tensor(test_mask, dtype=torch.long)
    
    return graph_data

# 4. Attention-based GNN Model
class AttentionGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, heads=4):
        super(AttentionGNN, self).__init__()
        self.conv1 = GATConv(input_dim, hidden_dim, heads=heads)
        self.conv2 = GATConv(hidden_dim * heads, hidden_dim, heads=heads)
        self.fc1 = nn.Linear(hidden_dim * heads, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.3)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = self.dropout(x)
        
        x = self.conv2(x, edge_index)
        x = F.elu(x)
        x = self.dropout(x)
        
        x = self.fc1(x)
        x = F.elu(x)
        x = self.dropout(x)
        
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# 5. Training Function
def train_model(model, data, optimizer, criterion, epochs=100):
    model.train()
    train_losses = []
    
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        
        train_losses.append(loss.item())
        
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item():.4f}')
    
    return train_losses

# 6. Evaluation Function
def evaluate_model(model, data, le_attack=None):
    model.eval()
    with torch.no_grad():
        pred = model(data).argmax(dim=1)
        
        # Get true and predicted labels
        y_true = data.y[data.test_mask].numpy()
        y_pred = pred[data.test_mask].numpy()
        
        # Classification report
        print("\nClassification Report:")
        if le_attack is not None:
            target_names = le_attack.classes_
        else:
            target_names = ['Benign', 'Malicious']
        print(classification_report(y_true, y_pred, target_names=target_names))
        
        # Confusion matrix
        cm = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title('Confusion Matrix')
        plt.savefig('confusion_matrix.png')
        plt.close()
        
        # ROC AUC for binary classification
        if len(np.unique(data.y)) == 2:
            probs = torch.softmax(model(data), dim=1)[:, 1]
            roc_auc = roc_auc_score(y_true, probs[data.test_mask].numpy())
            print(f"\nROC AUC Score: {roc_auc:.4f}")

# Main Execution
if __name__ == "__main__":
    # Load and preprocess data
    file_path = "../dataset/NF-BoT-IoT-V2.parquet"
    data, numerical_features, le_attack = load_and_preprocess_data(file_path)
    
    # Perform EDA
    perform_eda(data, numerical_features)
    
    # Binary Classification
    print("\n=== Binary Classification ===")
    binary_graph = create_graph_data(data, numerical_features, 'Label')
    
    # Initialize model
    binary_model = AttentionGNN(
        input_dim=len(numerical_features),
        hidden_dim=64,
        output_dim=2
    )
    
    # Training setup
    optimizer = torch.optim.Adam(binary_model.parameters(), lr=0.01)
    criterion = nn.NLLLoss()
    
    # Train model
    binary_losses = train_model(binary_model, binary_graph, optimizer, criterion)
    
    # Evaluate model
    evaluate_model(binary_model, binary_graph)
    
    # Multiclass Classification
    print("\n=== Multiclass Classification ===")
    multiclass_graph = create_graph_data(data, numerical_features, 'Attack_encoded')
    
    # Initialize model
    multiclass_model = AttentionGNN(
        input_dim=len(numerical_features),
        hidden_dim=64,
        output_dim=len(le_attack.classes_)
    )
    
    # Training setup
    optimizer = torch.optim.Adam(multiclass_model.parameters(), lr=0.01)
    criterion = nn.NLLLoss()
    
    # Train model
    multiclass_losses = train_model(multiclass_model, multiclass_graph, optimizer, criterion)
    
    # Evaluate model
    evaluate_model(multiclass_model, multiclass_graph, le_attack)
    
    # Plot training losses
    plt.figure(figsize=(10, 6))
    plt.plot(binary_losses, label='Binary Classification')
    plt.plot(multiclass_losses, label='Multiclass Classification')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss Curves')
    plt.legend()
    plt.savefig('training_loss.png')
    plt.close()

ModuleNotFoundError: No module named 'torch_geometric'

In [2]:
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import torch.nn.functional as F
import matplotlib.pyplot as plt
from collections import defaultdict
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv, GCNConv, GATConv
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Load 1 million rows from dataset
df = pd.read_csv("../dataset/NF_TON_IoT_V2/NF-ToN-IoT-v2.csv", nrows=1_000_000)
print(f"Loaded dataset shape: {df.shape}")

# Handle missing values and convert numerical columns to float64
for col in df.columns:
    if pd.api.types.is_numeric_dtype(df[col]):  # Check for any numeric type (int, float, etc.)
        df[col] = df[col].astype('float64').fillna(df[col].mean())
    else:
        df[col] = df[col].fillna(0)

# Encode categorical features
le_protocol = LabelEncoder()
df['PROTOCOL'] = le_protocol.fit_transform(df['PROTOCOL'].astype(str))
le_l7_proto = LabelEncoder()
df['L7_PROTO'] = le_l7_proto.fit_transform(df['L7_PROTO'].astype(str))

# Extract labels for binary classification
y = df['Label'].values

# Feature selection: exclude non-numerical columns and 'Label', 'Attack'
non_numerical_cols = ['IPV4_SRC_ADDR', 'IPV4_DST_ADDR']  # Common IP address columns
feature_cols = [col for col in df.columns if col not in ['Label', 'Attack'] + non_numerical_cols and pd.api.types.is_numeric_dtype(df[col])]
X = df[feature_cols].values
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Create preprocessed DataFrame
df_processed = pd.DataFrame(X, columns=feature_cols)
df_processed['Label'] = y
df_processed['L4_SRC_PORT'] = df['L4_SRC_PORT'].values
df_processed['L4_DST_PORT'] = df['L4_DST_PORT'].values
print(f"Preprocessed dataset shape: {df_processed.shape}")

# Graph construction
src_ports = df_processed['L4_SRC_PORT'].values
dst_ports = df_processed['L4_DST_PORT'].values
unique_ports = np.unique(np.concatenate([src_ports, dst_ports]))
port_to_idx = {port: idx for idx, port in enumerate(unique_ports)}

# Aggregate node features and labels
node_features = defaultdict(list)
node_labels = defaultdict(list)
for idx in range(len(df_processed)):
    row = df_processed.iloc[idx]
    src_port = row['L4_SRC_PORT']
    dst_port = row['L4_DST_PORT']
    features = row.drop(['Label', 'L4_SRC_PORT', 'L4_DST_PORT']).values
    label = row['Label']
    node_features[port_to_idx[src_port]].append(features[:-1])
    node_features[port_to_idx[dst_port]].append(features[:-1])
    node_labels[port_to_idx[src_port]].append(label)
    node_labels[port_to_idx[dst_port]].append(label)

# Compute average features and labels per node
x = []
y = []
for port_idx in range(len(unique_ports)):
    if port_idx in node_features:
        x.append(np.mean(node_features[port_idx], axis=0))
        y.append(np.mean(node_labels[port_idx]))
    else:
        x.append(np.zeros(df_processed.shape[1] - 3))
        y.append(0)

x = torch.tensor(x, dtype=torch.float)
y = torch.tensor(y, dtype=torch.long)

# Construct edges (bidirectional)
edge_index = []
for idx in range(len(df_processed)):
    row = df_processed.iloc[idx]
    src_idx = port_to_idx[row['L4_SRC_PORT']]
    dst_idx = port_to_idx[row['L4_DST_PORT']]
    edge_index.append([src_idx, dst_idx])
    edge_index.append([dst_idx, src_idx])

edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

# Create PyTorch Geometric Data object
data = Data(x=x, edge_index=edge_index, y=y)
print(f"Graph constructed: {data.num_nodes} nodes, {data.num_edges} edges")

# Create train/val/test masks (70% train, 10% val, 20% test)
num_nodes = data.num_nodes
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)
indices = np.random.permutation(num_nodes)
train_size = int(0.7 * num_nodes)
val_size = int(0.1 * num_nodes)
train_mask[indices[:train_size]] = True
val_mask[indices[train_size:train_size + val_size]] = True
test_mask[indices[train_size + val_size:]] = True
data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask

# Model definitions
class GraphSAGE(torch.nn.Module):
    def __init__(self):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(data.num_features, 64)
        self.conv2 = SAGEConv(64, 64)
        self.fc = torch.nn.Linear(64, 2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(data.num_features, 64)
        self.conv2 = GCNConv(64, 64)
        self.fc = torch.nn.Linear(64, 2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

class GAT(torch.nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
        self.conv1 = GATConv(data.num_features, 32, heads=2)
        self.conv2 = GATConv(32 * 2, 64, heads=1)
        self.fc = torch.nn.Linear(64, 2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = torch.nn.Linear(data.num_features, 64)
        self.fc2 = torch.nn.Linear(64, 64)
        self.fc3 = torch.nn.Linear(64, 2)

    def forward(self, data):
        x = data.x
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

# Training and evaluation function with early stopping
def train_and_evaluate(model, data, model_name, max_epochs=100, patience=10):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    best_val_loss = float('inf')
    patience_counter = 0
    best_model_state = None

    for epoch in range(max_epochs):
        model.train()
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        # Compute training metrics
        pred = out.argmax(dim=1)
        train_acc = accuracy_score(data.y[data.train_mask].numpy(), pred[data.train_mask].numpy())
        train_losses.append(loss.item())
        train_accuracies.append(train_acc)

        # Compute validation metrics
        model.eval()
        with torch.no_grad():
            out = model(data)
            val_loss = F.nll_loss(out[data.val_mask], data.y[data.val_mask])
            val_pred = out.argmax(dim=1)
            val_acc = accuracy_score(data.y[data.val_mask].numpy(), val_pred[data.val_mask].numpy())
            val_losses.append(val_loss.item())
            val_accuracies.append(val_acc)

        # Print metrics for every epoch
        print(f'{model_name} Epoch {epoch + 1}, Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}')

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f'{model_name} Early stopping at epoch {epoch + 1}')
                break

    # Load best model for evaluation
    model.load_state_dict(best_model_state)

    # Plot training vs validation accuracy and loss for this model
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(train_accuracies, label='Train Acc')
    plt.plot(val_accuracies, label='Val Acc')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title(f'{model_name} Training vs Validation Accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f'{model_name} Training vs Validation Loss')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f'{model_name}_acc_loss.png')
    plt.show()
    plt.close()

    # Evaluate on test set
    model.eval()
    with torch.no_grad():
        out = model(data)
        pred = out.argmax(dim=1)
        y_true = data.y[data.test_mask].numpy()
        y_pred = pred[data.test_mask].numpy()
        y_score = out[data.test_mask][:, 1].numpy()

        # Compute metrics
        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred, average='binary')
        recall = recall_score(y_true, y_pred, average='binary')
        f1 = f1_score(y_true, y_pred, average='binary')

        # Confusion matrix
        cm = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(6, 4))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title(f'{model_name} Confusion Matrix')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.savefig(f'{model_name}_confusion_matrix.png')
        plt.show()
        plt.close()

        # ROC curve
        fpr, tpr, _ = roc_curve(y_true, y_score)
        roc_auc = auc(fpr, tpr)
        plt.figure()
        plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'{model_name} ROC Curve')
        plt.legend(loc='lower right')
        plt.savefig(f'{model_name}_roc_curve.png')
        plt.show()
        plt.close()

    return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1, 'roc_auc': roc_auc,
            'train_losses': train_losses, 'val_losses': val_losses, 'train_accuracies': train_accuracies, 'val_accuracies': val_accuracies}

# Train and evaluate all models
models = {
    'GraphSAGE': GraphSAGE(),
    'GCN': GCN(),
    'GAT': GAT(),
    'MLP': MLP()
}
results = {}
for name, model in models.items():
    print(f"\nTraining {name}...")
    results[name] = train_and_evaluate(model, data, name)

# Model comparison plots (combined)
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
for name in models:
    plt.plot(results[name]['train_accuracies'], label=f'{name} Train')
    plt.plot(results[name]['val_accuracies'], '--', label=f'{name} Val')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy Comparison')
plt.legend()
plt.subplot(1, 2, 2)
for name in models:
    plt.plot(results[name]['train_losses'], label=f'{name} Train')
    plt.plot(results[name]['val_losses'], '--', label=f'{name} Val')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Comparison')
plt.legend()
plt.tight_layout()
plt.savefig('model_comparison.png')
plt.show()
plt.close()

# Print evaluation metrics
print("\nModel Comparison Metrics:")
for name in models:
    print(f"\n{name}:")
    print(f"Accuracy: {results[name]['accuracy']:.4f}")
    print(f"Precision: {results[name]['precision']:.4f}")
    print(f"Recall: {results[name]['recall']:.4f}")
    print(f"F1: {results[name]['f1']:.4f}")
    print(f"ROC AUC: {results[name]['roc_auc']:.4f}")

Loaded dataset shape: (1000000, 45)


c:\Users\gech\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1120: RuntimeWarning: overflow encountered in square
  temp **= 2
c:\Users\gech\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1126: RuntimeWarning: overflow encountered in square
  new_unnormalized_variance -= correction**2 / new_sample_count
c:\Users\gech\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1126: RuntimeWarning: invalid value encountered in subtract
  new_unnormalized_variance -= correction**2 / new_sample_count
c:\Users\gech\anaconda3\Lib\site-packages\sklearn\preprocessing\_data.py:81: RuntimeWarning: overflow encountered in square
  upper_bound = n_samples * eps * var + (n_samples * mean * eps) ** 2


Preprocessed dataset shape: (1000000, 42)


C:\Users\gech\AppData\Local\Temp\ipykernel_3348\1095886239.py:82: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  x = torch.tensor(x, dtype=torch.float)


Graph constructed: 65532 nodes, 2000000 edges

Training GraphSAGE...
GraphSAGE Epoch 1, Train Loss: nan, Val Loss: nan, Train Acc: 0.2922, Val Acc: 0.3026
GraphSAGE Epoch 2, Train Loss: nan, Val Loss: nan, Train Acc: 0.2922, Val Acc: 0.3026
GraphSAGE Epoch 3, Train Loss: nan, Val Loss: nan, Train Acc: 0.2922, Val Acc: 0.3026
GraphSAGE Epoch 4, Train Loss: nan, Val Loss: nan, Train Acc: 0.2922, Val Acc: 0.3026
GraphSAGE Epoch 5, Train Loss: nan, Val Loss: nan, Train Acc: 0.2922, Val Acc: 0.3026
GraphSAGE Epoch 6, Train Loss: nan, Val Loss: nan, Train Acc: 0.2922, Val Acc: 0.3026
GraphSAGE Epoch 7, Train Loss: nan, Val Loss: nan, Train Acc: 0.2922, Val Acc: 0.3026
GraphSAGE Epoch 8, Train Loss: nan, Val Loss: nan, Train Acc: 0.2922, Val Acc: 0.3026
GraphSAGE Epoch 9, Train Loss: nan, Val Loss: nan, Train Acc: 0.2922, Val Acc: 0.3026
GraphSAGE Epoch 10, Train Loss: nan, Val Loss: nan, Train Acc: 0.2922, Val Acc: 0.3026
GraphSAGE Early stopping at epoch 10


TypeError: Expected state_dict to be dict-like, got <class 'NoneType'>.